In [6]:
import pandas as pd
import os
# Define the root path
root_path = "/srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/spanish/cml_tts_dataset_spanish_v0.1"

# Set the output path based on your specific configuration
output_path = os.path.join(root_path)
path='/home/mcelylatorre/workspace/tmp'
# Path to your "train.csv" file
train_csv_path = os.path.join(root_path, "train.csv")

column_names = ['wav_filename','wav_filesize','transcript','transcript_wav2vec','levenshtein','duration','num_words','client_id', 'path']

df = pd.read_csv('/srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/spanish/cml_tts_dataset_spanish_v0.1/train.csv', delimiter='|')
df.columns = column_names



# Load the "train.csv" file into a DataFrame
# Construct full paths to the audio files using the "wav_filename" column
df['full_audio_path'] = df['wav_filename'].apply(lambda x: os.path.join(root_path, x))

print(df)



                                             wav_filename  wav_filesize  \
0          train/audio/11797/13165/11797_13165_000108.wav        642764   
1              train/audio/3946/6554/3946_6554_001421.wav        606282   
2              train/audio/3290/3105/3290_3105_000767.wav        597642   
3              train/audio/3946/6907/3946_6907_002742.wav        616842   
4          train/audio/10246/14144/10246_14144_000406.wav        629324   
...                                                   ...           ...   
168518    train/audio/3946/4644/3946_4644_000881-0001.wav        286124   
168519    train/audio/3946/4644/3946_4644_000881-0002.wav        251564   
168520    train/audio/3946/4644/3946_4644_000881-0003.wav        268844   
168521  train/audio/10206/10282/10206_10282_000108-000...        397484   
168522  train/audio/10206/10282/10206_10282_000108-000...        334124   

                                               transcript  \
0       Vengo a verte pasar todos los 

In [7]:
import os
from trainer import Trainer, TrainerArgs
from TTS.config import BaseAudioConfig, BaseDatasetConfig
from TTS.tts.configs.fast_speech_config import FastSpeechConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.forward_tts import ForwardTTS
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.tts.models.tacotron2 import Tacotron2
import torch

print(torch.cuda.is_available())
USE_CUDA = torch.cuda.is_available()

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.tacotron2_config import Tacotron2Config

output_path = "/srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/spanish/cml_tts_dataset_spanish_v0.1"

if not os.path.exists(output_path):
    os.makedirs(output_path)
    

# Create a BaseDatasetConfig object
dataset_config = BaseDatasetConfig(formatter="custom_formatter", meta_file_train="train.csv", path=os.path.join(output_path))

# Load the dataset using your custom formatter
train_samples, eval_samples = load_tts_samples([dataset_config], eval_split=True)

audio_config = BaseAudioConfig(
    sample_rate=24000,
    do_trim_silence=True,
    trim_db=23.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

# Configure your Tacotron2 model
config = Tacotron2Config(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    precompute_num_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    use_phonemes=True,
    phonemizer="gruut",
    phoneme_language="es-es",
    phoneme_cache_path=os.path.join(path, "phoneme_cache2"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=path,
    datasets=[dataset_config],
    use_speaker_embedding=True,
    min_text_len=0,
    max_text_len=500,
    min_audio_len=100000,  # Adjust this value based on your requirements
    max_audio_len=400000,  # Adjust this value based on your requirements
)

config.audio.fft_size = 2048
config.audio.win_length = 1200
config.audio.hop_length = 256
config.audio.num_mels = 80

ap = AudioProcessor.init_from_config(audio_config)

# Initialize the TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)



False
 | > Found 168523 files in /srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/spanish/cml_tts_dataset_spanish_v0.1
 > Setting up Audio Processor...
 | > sample_rate:24000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:23.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024


In [8]:
speaker_manager = SpeakerManager()
speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
config.num_speakers = speaker_manager.num_speakers

model = Tacotron2(config, ap, tokenizer, speaker_manager=speaker_manager)


 > Init speaker_embedding layer.


In [9]:
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 12
 | > Num. of Torch Threads: 6
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/home/mcelylatorre/workspace/tmp/run-November-06-2023_06+31PM-638afd4
/home/mcelylatorre/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 > Model has 32649330 parameters


 > `speakers.pth` is saved to /home/mcelylatorre/workspace/tmp/run-November-06-2023_06+31PM-638afd4/speakers.pth.
 > `speakers_file` is updated in the config.json.


In [ ]:
trainer.fit()